In [1]:
n = 1000  # number of neurons in one group
neuron_group_count = 100
R = 0.8  # ratio about excitory-inhibitory neurons

In [2]:
from brian2 import *
from tqdm import tqdm

%matplotlib inline


def draw_normals(n, start, stop):
    mu, sigma, numbers = start + (stop - start) / 2, (stop - start) / 6, zeros(n)
    for i in range(n):
        s = -1
        while (s < start) or (s > stop):
            s = numpy.random.normal(mu, sigma, 1)
            numbers[i] = s
    return numbers


eqs = Equations(
    """ 
    dv/dt = (0.04/ms/mV)*v**2 + (5/ms) * v + 140*mV/ms - u + I_syn/ms + I_in/ms : volt
    du/dt = a*((b*v) - u) : volt/second
    dx/dt = -x/(1*ms) : 1
    I_in = ceil(x)*((1/exp(1)))*amplitude : volt
    dI_syn/dt = - I_syn/tau : volt
    a : 1/second
    b : 1/second
    c : volt
    d : volt/second
    amplitude : volt
    tau : second
    """
)
# 公式サイトにあるIzhikevichモデルの例
# こちらでのvmがv、wが上の記法ではuになり、IがI_synとI_inに分かれている
# a = 0.02/ms; b = 0.2/ms
# eqs = '''dvm/dt = (0.04/ms/mV)*vm**2+(5/ms)*vm+140*mV/ms-w + I : volt
#          dw/dt = a*(b*vm-w) : volt/second
#          I : volt/second'''
# group = ... # see above

# reset specification of the Izhikevich model
reset = """
v = c
u += d
"""



# 各ニューロングループの生成
P = NeuronGroup(n * neuron_group_count, model=eqs, threshold="v=>30*mvolt", reset=reset, method="euler")
groups = []
# サブグループに分ける
for i in tqdm(range(neuron_group_count)):
    start = int(i * n)
    end = int((i + 1) * n)

    group = P[start:end]
    # 興奮性
    Pe = group[: int(n * R)]
    # Pe = P[start: start+int(n * R)]
    # 抑制性
    Pi = group[int(n * R) :]
    # Pi = P[start+int(neurons_in_group * R) :end]

    # 各種設定
    Pe.a = 0.02 / msecond
    Pe.b = 0.2 / msecond
    Pe.c = (15 * draw_normals(int(n * R), float(0), 1) - 65) * mvolt
    Pe.d = (-6 * draw_normals(int(n * R), float(0), 1) + 8) * mvolt / msecond
    Pe.tau = draw_normals(int(n * R), float(3), 15) * msecond

    Pi.a = (0.08 * draw_normals(n - int(n * R), float(0), 1) + 0.02) * 1 / msecond
    Pi.b = (-0.05 * draw_normals(n - int(n * R), float(0), 1) + 0.25) * 1 / msecond
    Pi.c = -65 * mvolt
    Pi.d = 2 * mvolt / msecond
    Pi.tau = draw_normals(n - int(n * R), float(3), 15) * msecond

    P.x = 0
    # 全く良くわからんがneuron_group_countをかける
    P.v = draw_normals(n * neuron_group_count, float(-50), float(-25)) * mvolt
    P.amplitude = draw_normals(n * neuron_group_count, 0, 8) * mvolt

    taupre = taupost = 20 * ms
    Apre = 0.01
    Apost = -Apre * taupre / taupost * 1.05

    # グループ内の接続
    # 興奮性ニューロン to 同じニューロングループ内の100個のニューロンへのランダム接続
    # Ce = Synapses(Pe, group, on_pre="I_syn+=1.5*mV")
    Ce = Synapses(
        Pe,
        group,
        """
             w : 1
             dapre/dt = -apre/taupre : 1 (event-driven)
             dapost/dt = -apost/taupost : 1 (event-driven)
             """,
        on_pre="""
             I_syn+=1.5*mV
             v_post += w
             apre += Apre
             w = clip(w+apost, 0, wmax)
             """,
        on_post="""
             apost += Apost
             w = clip(w+apre, 0, wmax)
             """,
    )
    Ce.connect(p=0.1)
    # 抑制性ニューロン　to 同じニューロングループ内の100個の興奮性ニューロンへのランダム接続
    Ci = Synapses(Pi, Pe, on_pre="I_syn-=8*mV")
    # Ci = Synapses(
    #     Pi,
    #     Pe,
    #     """
    #          w : 1
    #          dapre/dt = -apre/taupre : 1 (event-driven)
    #          dapost/dt = -apost/taupost : 1 (event-driven)
    #          """,
    #     on_pre="""
    #         I_syn-=8*mV
    #          v_post += w
    #          apre += Apre
    #          w = clip(w+apost, 0, wmax)
    #          """,
    #     on_post="""
    #          apost += Apost
    #          w = clip(w+apre, 0, wmax)
    #          """,
    # )
    Ci.connect(p=0.125)

    groups.append((group, Pe, Pi, Ce, Ci))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:55<00:00,  1.15s/it]


In [3]:
# WSモデルに従い、各グループの興奮性ニューロンから隣接する6つのノードへの接続と再配線を行う
import matplotlib.pyplot as pl
from smallworld import get_smallworld_graph
from smallworld.draw import draw_network

# define network parameters
N = neuron_group_count
k_over_2 = 3
# betas = [0, 0.025, 1.0]
betas = [1.0]
# labels = [r"$\beta=0$", r"$\beta=0.025$", r"$\beta=1$"]
labels = [r"$\beta=0$"]

focal_node = 0
for ib, beta in enumerate(betas):
    # generate small-world graphs and draw
    G = get_smallworld_graph(N, k_over_2, beta)

for edge in list(G.edges()):
    source_group_Pe = groups[edge[0]][1]
    target_group = groups[edge[1]][0]
    taupre = taupost = 20 * ms
    Apre = 0.01
    Apost = -Apre * taupre / taupost * 1.05
    Ce = Synapses(source_group_Pe, target_group, on_pre="I_syn+=1.5*mV")
    # Ce = Synapses(
    #     source_group_Pe,
    #     target_group,
    #     """
    #          w : 1
    #          dapre/dt = -apre/taupre : 1 (event-driven)
    #          dapost/dt = -apost/taupost : 1 (event-driven)
    #          """,
    #     on_pre="""
    #          I_syn+=1.5*mV
    #          v_post += w
    #          apre += Apre
    #          w = clip(w+apost, 0, wmax)
    #          """,
    #     on_post="""
    #          apost += Apost
    #          w = clip(w+apre, 0, wmax)
    #          """,
    # )
    # このパラメータは間違い
    # Ce.connect(p=0.00000375)
    # 各興奮性ニューロンが、他のニューロングループと三本の接続を持つので、接続する確率は3/1000
    Ce.connect(p=0.003)

In [ ]:
from collections import defaultdict
# とりあええず1000秒動かす
run_time_ms = 1000*1000

# statemonitorを1つだけにするver
get_value_interval_ms = 1
run_time_ms = 100*1000
time_count = int(run_time_ms/get_value_interval_ms)

defaultclock.dt = get_value_interval_ms*ms

V = StateMonitor(P, "v", record=True)
run(run_time_ms*ms)

lap =defaultdict(list) 
for i in tqdm(range(neuron_group_count)):
    start = int(i * n)
    end = int((i + 1) * n)

    group = V.v[start:end]
    # 興奮性
    Pe = group[: int(n * R)]
    for j in range(time_count):
        data = [neuron[j]/mV for neuron in list(group)]
        mean = sum(data)/len(data)
        lap[i].append(mean)

  2%|████▏                                                                                                                                                                                                               | 2/100 [39:32<32:25:44, 1191.27s/it]

In [ ]:
len(lap[0])

In [ ]:
import neurokit2 as nk
import numpy as np
import pandas as pd
# 全てのニューロングループに対してMSEの計算を行う
results = []
for i in range(neuron_group_count):
    result = nk.entropy_multiscale(signal=np.array(lap[i]),scale=40,dimension=1)
    results.append(result[1]['Values'])
    
pd.DataFrame(results).to_csv('test.csv',index=False)

# 今使ってないやつ↓↓

In [ ]:
# ↓statemonitorを複数仕掛けてしまい、最後のものしか値を取っていなかったやつ
get_value_interval_ms = 0.1
run_time_ms = 1
time_count = int(run_time_ms/get_value_interval_ms)


defaultclock.dt = get_value_interval_ms*ms

# グループ毎にstatemonitorを仕掛ける
state_monitors = []
for i in range(neuron_group_count):
    V = StateMonitor(groups[i][1], "v", record=True)
    state_monitors.append(V)
    
run(run_time_ms*ms)

lap =defaultdict(dict) 
# LAPをニューロングループ毎,0.1msごとに出す
for i,ne_neuron_group in enumerate(state_monitors):
    # print(ne_neuron_group)
    for j in range(time_count):
        for neuron in list(ne_neuron_group.v):
            print(neuron)
            print(j)
            print(neuron[j]/mV)
            
            # data = [neuron[j]/mV for neuron in list(ne_neuron_group.v)]
            # mean = sum(data)/len(data)
            # lap[i][j] = mean

In [ ]:
# state_monitors[9].v

In [ ]:
# state_monitors[0].v

In [ ]:
state_monitors[0].v[0]/mV

In [ ]:
V = StateMonitor(groups[9][1], "v", record=True)
run(1*ms)

In [ ]:
list(V.v)[0][0]/mV

In [ ]:
int(V.v[0]/mV)

In [ ]:
%matplotlib inline

defaultclock.dt = 0.1*ms
# 6th: Run & monitor
V = StateMonitor(P, "v", record=True)
run(0.1*ms)

# plot(V.t/ms,V.v[0])

In [ ]:
len(list(V.v))

In [ ]:
P.state

In [ ]:
V

In [ ]:
len(list(V.v))

In [ ]:
%matplotlib inline
# 6th: Run & monitor
# V = StateMonitor(groups[0][0], "v", record=True)
V = StateMonitor(groups[0][1], "v", record=True)
run(1*ms)

a = list(V.v)
b = list(V.v[0])
# plot(V.t/ms,V.v[0])
# plot(V.t/ms,V.v)

In [ ]:
# V.vで、そのグループの全てのニューロンという意味になる
# ただし、1つ1つのニューロンが10個の計測値を持っている
len(a)

In [ ]:
# V.v[0]で、ある一つのニューロンという意味になる

In [ ]:
len(b)

In [ ]:
a[0]

In [ ]:
V.v[0]

In [ ]:
# plot(V.t/ms,V.v)

In [ ]:
# 0.1ms毎に取得した合計10個の膜電位の値を可視化
# 何故か2ms~3msまでの期間になっている。runを三回やったからかな？
plot(V.t/ms,V.v[0])

In [ ]:
V.t

In [ ]:
len(a)

In [ ]:
run(1000 * ms)
subplot(211)
plot(V.t[:200] / ms, V.v[0][:200] / mV, "r")
plot(V.t[:200] / ms, V.v[100][:200] / mV, "g")
plot(V.t[:200] / ms, V.v[200][:200] / mV, "b")
show()

In [ ]:
%matplotlib inline
# 6th: Run & monitor
M = SpikeMonitor(P)
V = StateMonitor(P, "v", record=True)
run(1000 * ms)
subplot(211)
plot(M.t / ms, M.i, ".")
subplot(212)
plot(V.t[:200] / ms, V.v[0][:200] / mV, "r")
plot(V.t[:200] / ms, V.v[100][:200] / mV, "g")
plot(V.t[:200] / ms, V.v[200][:200] / mV, "b")
show()

In [ ]:
def get_neighbor_groups(index):
    indexes = [i for i in range(100)]
    if 3 <= index and index <= 96:
        return indexes[index - 3 : index] + indexes[index + 1 : index + 4]
    if index < 3:
        return indexes[index - 3 :] + indexes[:index] + indexes[index + 1 : index + 4]
    if 96 < index and index <= 99:
        return indexes[index - 3 : index] + indexes[index + 1 : 100] + indexes[: index - 96]
    if 99 < index:
        raise ValueError("out of index")

In [ ]:
# I_in = ceil(x)*(x>(1/exp(1)))*amplitude : volt

In [ ]:
from brian2 import *

%matplotlib inline


def izhikevich_model():
    def draw_normals(n, start, stop):
        mu, sigma, numbers = start + (stop - start) / 2, (stop - start) / 6, zeros(n)
        for i in range(n):
            s = -1
            while (s < start) or (s > stop):
                s = numpy.random.normal(mu, sigma, 1)
                numbers[i] = s
        return numbers

    n = 2000  # number of neurons
    R = 0.8  # ratio about excitory-inhibitory neurons

    eqs = Equations(
        """ 
    dv/dt = (0.04/ms/mV)*v**2 + (5/ms) * v + 140*mV/ms - u + I_syn/ms + I_in/ms : volt
    du/dt = a*((b*v) - u) : volt/second
    dx/dt = -x/(1*ms) : 1
    I_in = ceil(x)*((1/exp(1)))*amplitude : volt
    dI_syn/dt = - I_syn/tau : volt
    a : 1/second
    b : 1/second
    c : volt
    d : volt/second
    amplitude : volt
    tau : second
    """
    )
    # 公式サイトにあるIzhikevichモデルの例
    # こちらでのvmがv、wが上の記法ではuになり、IがI_synとI_inに分かれている
    # a = 0.02/ms; b = 0.2/ms
    # eqs = '''dvm/dt = (0.04/ms/mV)*vm**2+(5/ms)*vm+140*mV/ms-w + I : volt
    #          dw/dt = a*(b*vm-w) : volt/second
    #          I : volt/second'''
    # group = ... # see above

    # reset specification of the Izhikevich model
    reset = """
    v = c
    u += d
    """

    # 2nd: Define the Population of Neurons P
    P = NeuronGroup(n, model=eqs, threshold="v>30*mvolt", reset=reset, method="euler")

    # 3rd: Define subgroups of the neurons (regular spiking/fast spiking)
    # inhibitorの方は、おそらくパルブアルブミン型の抑制性ニューロンを表現している
    # https://knowingneurons.com/2014/11/05/inhibitory-neurons-keeping-the-brains-traffic-in-check/
    Pe = P[: int(n * R)]
    Pi = P[int(n * R) :]

    # 4th: initialize starting neuronal p"""!!!<<<nicht wie im Paper>>>!!!"""arameters for the simulation
    # この辺の数字とかは論文に合わせる必要がある。論文ではIzhikevichの論文で使われている設定そのままとのこと。
    # この辺参考になる
    # https://qiita.com/arakiii/items/7522c5a1b3427bd51259
    # https://compneuro-julia.github.io/neuron-model/izhikevich.html
    # http://gaya.jp/spiking_neuron/izhikevich.htm
    Pe.a = 0.02 / msecond
    Pe.b = 0.2 / msecond
    Pe.c = (15 * draw_normals(int(n * R), float(0), 1) - 65) * mvolt
    Pe.d = (-6 * draw_normals(int(n * R), float(0), 1) + 8) * mvolt / msecond
    Pe.tau = draw_normals(int(n * R), float(3), 15) * msecond
    Pi.a = (0.08 * draw_normals(n - int(n * R), float(0), 1) + 0.02) * 1 / msecond
    Pi.b = (-0.05 * draw_normals(n - int(n * R), float(0), 1) + 0.25) * 1 / msecond
    Pi.c = -65 * mvolt
    Pi.d = 2 * mvolt / msecond
    Pi.tau = draw_normals(n - int(n * R), float(3), 15) * msecond
    P.x = 0
    P.v = draw_normals(n, float(-50), float(-25)) * mvolt
    P.amplitude = draw_normals(n, 0, 8) * mvolt

    # 5th: Connect synapses
    Ce = Synapses(Pe, P, on_pre="I_syn+=1.5*mV")
    Ce.connect(p=0.05)
    Ci = Synapses(Pi, P, on_pre="I_syn-=8*mV")
    Ci.connect(p=0.05)

    # 6th: Run & monitor
    M = SpikeMonitor(P)
    V = StateMonitor(P, "v", record=True)
    run(500 * ms)
    subplot(211)
    plot(M.t / ms, M.i, ".")
    subplot(212)
    plot(V.t[:200] / ms, V.v[0][:200] / mV, "r")
    plot(V.t[:200] / ms, V.v[100][:200] / mV, "g")
    plot(V.t[:200] / ms, V.v[200][:200] / mV, "b")
    show()


izhikevich_model()

In [ ]:
import matplotlib.pyplot as pl
from smallworld import get_smallworld_graph
from smallworld.draw import draw_network

# define network parameters
N = 100
k_over_2 = 3
betas = [0, 0.025, 1.0]
labels = [r"$\beta=0$", r"$\beta=0.025$", r"$\beta=1$"]

focal_node = 0

fig, ax = pl.subplots(1, 3, figsize=(9, 3))


# scan beta values
for ib, beta in enumerate(betas):

    # generate small-world graphs and draw
    G = get_smallworld_graph(N, k_over_2, beta)
    draw_network(G, k_over_2, focal_node=focal_node, ax=ax[ib])

    ax[ib].set_title(labels[ib], fontsize=11)

# show
pl.subplots_adjust(wspace=0.3)
pl.show()

In [ ]:
import networkx as nx

nx.info(G)

In [ ]:
G.nodes()

In [ ]:
G.edges()

In [ ]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), "ok", ms=10)
    plot(ones(Nt), arange(Nt), "ok", ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], "-k")
    xticks([0, 1], ["Source", "Target"])
    ylabel("Neuron index")
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, "ok")
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel("Source neuron index")
    ylabel("Target neuron index")


# visualise_connectivity(S)